In [ ]:
# installing the project (kidney_dtsim) in editable mode from the repository root
%pip install -e ".."

In [ ]:
# Data visualization tools.
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("whitegrid")

# Has multi-dimensional arrays and matrices. Has a large collection of
# mathematical functions to operate on these arrays.
import numpy as np

from scipy.stats import gaussian_kde

# Data manipulation and analysis.
import pandas as pd

# for survival analysis
from lifelines import CoxPHFitter

# to be able to clear output
from IPython.display import clear_output

# Enables autoreloading of specified modules.
%load_ext autoreload 
%autoreload 1

# import the allocation model
from kidney_dtsim.model import *
from kidney_dtsim.helper_functions import *

# specify model to be autoreloaded
%aimport kidney_dtsim

data_path = "../model_input_public/"

Aim of this script is to simulate the outcome of a different age border for the entry into the ESP program.


# Loading the data

We run the notebook 'model_load_display_input.ipynb' to load the model input.

In [ ]:
%run model_load_input.ipynb

clear_output()

# Running the model with multiprocessing

The model is now initialized. Multiprocessing is used to run simulations in parallel.

In [ ]:
step_size= 1/36 # in fractions of a year
years_to_simulate = 11
steps = 1+ int(years_to_simulate / step_size)

# these parameters are kept constant across all model runs
constant_params = {
    "df_init_patients": df_active_waiting_2006, 
    "n_pts_add_per_step": distribute_patients_evenly(new_pts_list, step_size),
    "n_organ_donors_step": distribute_patients_evenly(tx_per_year_list, step_size),
    "pts_age_list": list_pts_age,
    "donor_age_list": list_donor_age,
    "vpra_df": df_peak_vpra,
    "removal_model": cph_spline,
    "df_nt_status_prob": df_nt_status_prob,
    "dict_log_odds_ETKAS": dict_log_odds_ETKAS,
    "dict_log_odds_ESP": dict_log_odds_ESP,
    "df_dial_to_reg_time": df_dial_to_reg,
    "hla_a_freq_dict": hla_a_freq,
    "hla_b_freq_dict": hla_b_freq,
    "hla_dr_freq_dict": hla_dr_freq,
    "hla_hapl_freq_df": df_hla_haplo_freq,
    "donor_reg_freq_dict": donor_region_freq,
    "recipient_reg_freq_dict": recipient_region_freq,
    "living_donor_prob_dict": living_donor_prob,
    "donor_blood_grp_freq_dict": donor_blood_grp_freq_dict,
    "recipient_blood_grp_freq_dict": recipient_blood_grp_freq_dict,
    "max_rank_pts_organ_offer": 1000,
    "et_groups_bool": True,
    "esp_national_allo": False,
    "extended_allo": True,
    "prob_stay_in_etkas": 0.36,
    "step_size": step_size,
    "datacollector_interval": int(1/step_size),
    "years_to_simulate": years_to_simulate
}

# these parameters are varying in between model runs
variable_params = [
    {"seed": e1,
     "esp_age_border": e2} for e1 in range(0,5) for e2 in [65, 999]
]

combined_patient_donor_data, combined_model_data = batch_run(constant_params, variable_params)
clear_output()

## Summary

In [ ]:
combined_model_data = combined_model_data.reset_index(names="step")

In [ ]:
mult_model_summary = combined_model_data.groupby(["model_esp_age_border", "step"]).agg(["mean", "std"]).drop(columns=["model_run_id", "model_seed"])

mult_model_summary = mult_model_summary.round(1)
mult_model_summary

## Exporting Data

The data is exported to be analyzed in a R notebook.

In [ ]:
combined_model_data.to_csv("../res/model_results.csv")
combined_patient_donor_data.to_csv("../res/model_results_merged_patients_donor.csv")